In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data
from functions.plot_angle_gradient import plot_angle_gradient

In [ ]:
# Open data files
from functions.load_pickles import load_pickles
stokes_list, _ = load_pickles(select='stokes')

In [ ]:
# Extract each Stokes parameter into dictionary, to make it easier to work with
I = stokes_list['I']
Q = stokes_list['Q']
U = stokes_list['U']
V = stokes_list['V']

In [ ]:
# Constants
kB = 1.3806488e-16 # [erg K-1]
h = 6.6260755e-27  # [erg s]
c = 2.99792458e10  # [cm · s−1]
lambda0  = [6301.51*1e-8, 6302.50*1e-8]   # Angstroms to cm
l = np.array([lambda0[0], np.mean(lambda0), lambda0[1]])
Teff = 5780 # [K] T quiet sun average

In [ ]:
# Plot the spectrum with calibrated wavelengths, adding vertical lines to represent the four points selected for representation
plt.figure(figsize=(8,5))
plt.plot(I.wave_array, I.mean_quiet)
plt.xlabel('Calibrated wavelength [nm]')

# Select 4 frames leading up to the absorption line (each frame represents rising through the photosphere)
selected_frames = np.array([29, 55, 75])
plt.vlines(I.wave_array[selected_frames-1], I.mean_quiet.min(), I.mean_quiet.max(), colors='purple', ls='--', lw=1,)
plt.vlines(I.wave_array[selected_frames+1], I.mean_quiet.min(), I.mean_quiet.max(), colors='purple', ls='--', lw=1,)
plt.show()

In [ ]:
T_select = np.empty(np.shape(I.data[:,:,0]))
frame=56

print(f"Calculating temperature from {I.wave_array[frame-1]} to {I.wave_array[frame+1]} Angstroms")
T_select[:,:] = (1/Teff - kB*l[0]/h/c * np.log(I.data_n[:,:,frame-1:frame+2].mean(axis=2)))**-1

In [ ]:
T = np.empty(np.shape(I.data[:,:,:]))
for i in range(I.size_wave):
    T[:,:,:] = (1/Teff - kB*I.wave_array[i]*1e-8/h/c * np.log(I.data_n[:,:,:]))**-1

In [ ]:
plot_data(T[:,:,0], title="T frame 0")
plot_data(T[:,:,26], title="T frame 26")


In [ ]:
# Plot all 4 temperature maps as subplots in the same figure
fig, axs = plt.subplots(1, 3, figsize=(12, 6))

# Titles and scales for each subplot
titles = [
    "Temperature in the line core, first line",
    "Temperature in the continuum",
    "Temperature in the line core, second line"
]
scales = [
    [T[:,:,0:2].min(), T[:,:,0:2].max()],
    [T[:,:,0:2].min(), T[:,:,0:2].max()],
    [T[:,:,2:4].min(), T[:,:,2:4].max()],
    [T[:,:,2:4].min(), T[:,:,2:4].max()]
]

for i in range(3):
    ax = axs[i]
    im = ax.imshow(T_select[:,:,i], vmin=scales[i][0], vmax=scales[i][1], cmap='inferno', origin='lower')
    ax.set_title(titles[i])
    ax.set_xlabel('X')

# TODO: fix the third image! it's smaller for some reason

axs[0].set_ylabel('Y')
fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04, label='Temperature [K]')
plt.tight_layout()
plt.show()


In [ ]:
# Plot all 4 temperature maps as subplots in the same figure

plot_data(T_select[:,:], colourmap='magma', colourbar_label='Temperature [K]')

In [ ]:
# Plot all 4 temperature maps as subplots in the same figure
fig, axs = plt.subplots(1, 2, figsize=(9, 6))

# Titles and labels for each subplot
titles = [
    "Intensity in the continuum",
    "Temperature in the continuum",
]

labels = [
    "Intensity (normalised to 1)",
    "Temperature [K]",
]

data1 = I.data_n[:,:,selected_frames[1]-1:selected_frames[1]+2].mean(axis=2)

for i in range(2):
    ax = axs[i]
    im = ax.imshow(T_select[:,:,1] if i == 1 else data1, cmap='inferno')
    ax.set_title(titles[i])
    ax.set_xlabel('X')
    fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04, label=labels[i])


axs[0].set_ylabel('Y')
plt.tight_layout()
plt.show()


In [ ]:
print(f'Max temperature: {T_select[:,:,1].max()} K')
print(f'Min temperature: {T_select[:,:,1].min()} K')

# Select pixels of interest
x_pix = np.array([500, 180, 290, 400, 320, 300])
y_pix = np.array([118, 113, 538, 354, 428, 413])

for i in range(np.size(x_pix)):
    print(f'Pixel ({x_pix[i]}, {y_pix[i]}): T = {T_select[y_pix[i], x_pix[i], 1]:.2f} K')

In [ ]:
A & (500, 118) & 5793.67 \\
B & (180, 113) & 5520.89  \\
C & (400, 354) & 5169.65 \\
D & (290, 538) & 5550.55 \\
E & (320, 428) & 3544.92 \\
F & (300, 413) & 3715.14

In [ ]:
# Make rectangle around them, for better averaging
xmin = x_pix - 3
xmax = x_pix + 3
ymin = y_pix - 3
ymax = y_pix + 3
xwidth = xmax - xmin
ywidth = ymax - ymin

# Plot intensity data
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 8))
img = ax.imshow(T_select[:,:,1], cmap='cool', origin='lower')
ax.set_axis_off()

# Create rectangle patches and add the patch to the Axes
for i in range(6):
    ax.add_patch(Rectangle((xmin[i], ymin[i]), xwidth[i], ywidth[i], linewidth=2, edgecolor='r', facecolor='none'))


# Save figure to file
# fig.savefig("generated/polarisation_selection_square.png", dpi=150)
# print("Saved figure to file", "generated/polarisation_selection_square.png")

In [ ]:
# Plot profiles of each pixel
pixel_titles = [f'Quiet sun', f'Exterior to  sunspot', f'Penumbra 1', f'Penmbra 2', f'Umbra 1', f'Umbra 2']
fig, axs = plt.subplots(1,1, figsize=(12,7))
fig.suptitle('Temperature variation for selected areas', fontsize=20)

# Choose a colormap
num_lines = 6
cmap = plt.get_cmap('turbo')
colors = [cmap(i / num_lines) for i in range(num_lines)]

for j in range(num_lines):
    axs.plot(I.wave_array, T[ymin[j]:ymax[j],xmin[j]:xmax[j],:].mean(axis=(0,1)), linestyle='-', label=pixel_titles[j], color=colors[j])
    axs.set_xlabel(r'[$\AA$]')
    axs.set_ylabel('[K]')
    # axs[row, col].vlines(line_wavelengths, stokes_list[param].data_n[y_pix[j],x_pix[j],:].min(), stokes_list[param].data_n[y_pix[j],x_pix[j],:].max(), colors='gray', linestyles='dashed')
    # axs[i, j].legend()


axs.legend()


In [ ]:
# Select pixels of interest
plate_scale_x = 0.14857 # arcseconds per pixel
plate_scale_y = 0.16 # arcseconds per pixel
x_pix = np.array([500, 180, 290, 400, 320, 300])
y_pix = np.array([118, 113, 538, 354, 428, 413])
# Plot polarization and selected points
fig, ax, img = plot_data(nova.mp[:,:,0], colourmap='gist_earth', title='Selected pixels the polarisation map')
fig.show()
ax.scatter(x_pix*plate_scale_x, y_pix*plate_scale_y, facecolors='none', edgecolors='red')

# TODO: adapt this to temp
# TODO: https://medium.com/@marvelouskgc/three-ways-to-add-labels-to-each-data-point-in-a-scatter-plot-in-python-matplotlib-eugene-tsai-42e4094dc07e

In [ ]:
# TODO: save temperature to stokes pickle7
